In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [2]:
mb_size = 32
X_dim = 784
z_dim = 10
h_dim = 128
lam = 10
n_disc = 5
lr = 1e-4

mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def G(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob


def D(X):
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    out = tf.matmul(D_h1, D_W2) + D_b2
    return out

In [4]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


z = tf.placeholder(tf.float32, shape=[None, z_dim])

G_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [ ]:
G_sample = G(z)
D_real = D(X)
D_fake = D(G_sample)

eps = tf.random_uniform([mb_size, 1], minval=0., maxval=1.)
X_inter = eps*X + (1. - eps)*G_sample
grad = tf.gradients(D(X_inter), [X_inter])[0]
grad_norm = tf.sqrt(tf.reduce_sum((grad)**2, axis=1))
grad_pen = lam * tf.reduce_mean((grad_norm - 1)**2)

D_loss = tf.reduce_mean(D_fake) - tf.reduce_mean(D_real) + grad_pen
G_loss = -tf.reduce_mean(D_fake)

D_solver = (tf.train.AdamOptimizer(learning_rate=lr, beta1=0.5)
            .minimize(D_loss, var_list=theta_D))
G_solver = (tf.train.AdamOptimizer(learning_rate=lr, beta1=0.5)
            .minimize(G_loss, var_list=theta_G))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0
with tf.device("/gpu:0"):
    for it in range(1000000):
        for _ in range(n_disc):
            X_mb, _ = mnist.train.next_batch(mb_size)

            _, D_loss_curr = sess.run(
                [D_solver, D_loss],
                feed_dict={X: X_mb, z: sample_z(mb_size, z_dim)}
            )

        _, G_loss_curr = sess.run(
            [G_solver, G_loss],
            feed_dict={z: sample_z(mb_size, z_dim)}
        )

        if it % 1000 == 0:
            print('Iter: {}; D loss: {:.4}; G_loss: {:.4}'
                  .format(it, D_loss_curr, G_loss_curr))

            if it % 1000 == 0:
                samples = sess.run(G_sample, feed_dict={z: sample_z(16, z_dim)})

                fig = plot(samples)
                plt.savefig('out/{}.png'
                            .format(str(i).zfill(3)), bbox_inches='tight')
                i += 1
                plt.close(fig)

Iter: 0; D loss: -0.6011; G_loss: 1.845
Iter: 1000; D loss: -3.585; G_loss: 0.2246
Iter: 2000; D loss: -2.473; G_loss: -0.5106
Iter: 3000; D loss: -2.021; G_loss: -0.6321
Iter: 4000; D loss: -1.947; G_loss: -0.7018
Iter: 5000; D loss: -2.162; G_loss: -0.6881
Iter: 6000; D loss: -1.93; G_loss: -0.878
Iter: 7000; D loss: -1.714; G_loss: -1.021
Iter: 8000; D loss: -1.491; G_loss: -1.011
Iter: 9000; D loss: -1.824; G_loss: -0.5309
Iter: 10000; D loss: -1.625; G_loss: 0.02787
Iter: 11000; D loss: -2.115; G_loss: -0.007266
Iter: 12000; D loss: -1.906; G_loss: 0.2681
Iter: 13000; D loss: -2.032; G_loss: 0.1682
Iter: 14000; D loss: -1.858; G_loss: 0.2463
Iter: 15000; D loss: -2.12; G_loss: 0.712
Iter: 16000; D loss: -1.772; G_loss: 1.103
Iter: 17000; D loss: -1.801; G_loss: 1.138
Iter: 18000; D loss: -1.846; G_loss: 1.253
Iter: 19000; D loss: -1.736; G_loss: 1.537
Iter: 20000; D loss: -1.965; G_loss: 1.139
Iter: 21000; D loss: -1.694; G_loss: 1.138
Iter: 22000; D loss: -1.611; G_loss: 1.138
It